In [2]:
!ipip insstall spark

/bin/bash: line 1: ipip: command not found


In [4]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=a2eb16307d2abea3ae6eb3d6fec7254f200af9d6e64880776a9849ea2f7322d0
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [5]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
spark = SparkSession.builder.config("spark.driver.memory", "8g").getOrCreate()


In [12]:
data = [['male', 1, None], ['female', 2, 4], ['male', 3, 2]]
data_spark = spark.createDataFrame(data,['Gender','Stt', 'Id'])
data_spark.show()

+------+---+----+
|Gender|Stt|  Id|
+------+---+----+
|  male|  1|NULL|
|female|  2|   4|
|  male|  3|   2|
+------+---+----+



In [14]:
data_spark.select('Gender').show()

+------+
|Gender|
+------+
|  male|
|female|
|  male|
+------+



In [23]:
data_spark.printSchema()

root
 |-- Gender: string (nullable = true)
 |-- Stt: long (nullable = true)
 |-- Id: long (nullable = true)



In [21]:
data_spark.fill(NULL,['Id'])

AttributeError: 'DataFrame' object has no attribute 'fill'

In [34]:
data_spark.show()

+------+---+----+
|Gender|Stt|  Id|
+------+---+----+
|  male|  1|NULL|
|female|  2|   4|
|  male|  3|   2|
+------+---+----+



In [38]:
mapping = {
    'Gender': '4',
    'Stt': '3'
}
data_spark.columns

['Gender', 'Stt', 'Id']

In [39]:
new_names = [mapping.get(col,col) for col in data_spark.columns]
new_names

['4', '3', 'Id']

In [40]:
data_spark.toDF(*new_names).show(4)

+------+---+----+
|     4|  3|  Id|
+------+---+----+
|  male|  1|NULL|
|female|  2|   4|
|  male|  3|   2|
+------+---+----+



In [41]:
data_spark.withColumnRenamed('Newspaper','Paper').show(4)

+------+---+----+
|Gender|Stt|  Id|
+------+---+----+
|  male|  1|NULL|
|female|  2|   4|
|  male|  3|   2|
+------+---+----+



In [42]:
#add column
data_spark = data_spark.withColumn('Sum', col('Stt') + col('Id'))


In [43]:
data_spark.show()

+------+---+----+----+
|Gender|Stt|  Id| Sum|
+------+---+----+----+
|  male|  1|NULL|NULL|
|female|  2|   4|   6|
|  male|  3|   2|   5|
+------+---+----+----+



In [44]:
data_spark = data_spark.withColumn('Description', col('Stt') + 'okkk') \
                       .withColumn('GenDescription', col('Gender') + 'okkk') \
                       .withColumn('Test1', monotonically_increasing_id()) \
                       .withColumn('Test2', monotonically_increasing_id())

data_spark.show()


+------+---+----+----+-----------+--------------+----------+----------+
|Gender|Stt|  Id| Sum|Description|GenDescription|     Test1|     Test2|
+------+---+----+----+-----------+--------------+----------+----------+
|  male|  1|NULL|NULL|       NULL|          NULL|         0|         0|
|female|  2|   4|   6|       NULL|          NULL|8589934592|8589934592|
|  male|  3|   2|   5|       NULL|          NULL|8589934593|8589934593|
+------+---+----+----+-----------+--------------+----------+----------+



In [45]:
#drop column
data_spark = data_spark.drop('Test2')
data_spark.show()

+------+---+----+----+-----------+--------------+----------+
|Gender|Stt|  Id| Sum|Description|GenDescription|     Test1|
+------+---+----+----+-----------+--------------+----------+
|  male|  1|NULL|NULL|       NULL|          NULL|         0|
|female|  2|   4|   6|       NULL|          NULL|8589934592|
|  male|  3|   2|   5|       NULL|          NULL|8589934593|
+------+---+----+----+-----------+--------------+----------+



### Working with csv file

In [47]:
csv_file_path = '/content/drive/My Drive/Data/Advertising.csv'
ds = spark.read.csv(csv_file_path, header=True)
ds.show(10)

+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3|  9.3|
|151.5| 41.3|     58.5| 18.5|
|180.8| 10.8|     58.4| 12.9|
|  8.7| 48.9|       75|  7.2|
| 57.5| 32.8|     23.5| 11.8|
|120.2| 19.6|     11.6| 13.2|
|  8.6|  2.1|        1|  4.8|
|199.8|  2.6|     21.2| 10.6|
+-----+-----+---------+-----+
only showing top 10 rows



In [48]:
#filter
ds[ds.Newspaper<20].show()

+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|120.2| 19.6|     11.6| 13.2|
|  8.6|  2.1|        1|  4.8|
|214.7|   24|        4| 17.4|
| 97.5|  7.6|      7.2|  9.7|
| 69.2| 20.5|     18.3| 11.3|
|147.3| 23.9|     19.1| 14.6|
| 62.3| 12.6|     18.3|  9.7|
|262.9|  3.5|     19.5|   12|
|142.9| 29.3|     12.6|   15|
|265.6|   20|      0.3| 17.4|
| 95.7|  1.4|      7.4|  9.5|
|290.7|  4.1|      8.5| 12.8|
|266.9| 43.8|        5| 25.4|
|293.6| 27.7|      1.8| 20.7|
|239.9| 41.5|     18.5| 23.2|
|100.4|  9.6|      3.6| 10.7|
|262.7| 28.8|     15.9| 20.2|
|136.2| 19.2|     16.6| 13.2|
|210.7| 29.5|      9.3| 18.4|
|102.7| 29.6|      8.4|   14|
+-----+-----+---------+-----+
only showing top 20 rows



In [49]:
ds.count()

200

In [50]:
ds[(ds.Newspaper > 20) & (ds.TV>100)].show()

+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
|151.5| 41.3|     58.5| 18.5|
|180.8| 10.8|     58.4| 12.9|
|199.8|  2.6|     21.2| 10.6|
|204.1| 32.9|       46|   19|
|195.4| 47.7|     52.9| 22.4|
|281.4| 39.6|     55.8| 24.4|
|218.4| 27.7|     53.4|   18|
|237.4|  5.1|     23.5| 12.5|
|228.3| 16.9|     26.2| 15.5|
|240.1| 16.7|     22.9| 15.9|
|248.8| 27.1|     22.9| 18.9|
|292.9| 28.3|     43.2| 21.4|
|112.9| 17.4|     38.6| 11.9|
|  228| 37.7|       32| 21.5|
|202.5| 22.3|     31.6| 16.6|
|  177| 33.4|     38.7| 17.1|
|206.9|  8.4|     26.4| 12.9|
|175.1| 22.5|     31.5| 14.9|
|227.2| 15.8|     49.9| 14.8|
+-----+-----+---------+-----+
only showing top 20 rows



In [51]:
import pyspark.sql.functions as F

In [55]:
# Add new column
ds = ds.withColumn('cond',
                   F.when((ds.TV > 100) & (ds.Radio < 40), 1)
                    .when(ds.Sales > 10, 2)
                    .otherwise(3))
ds.show()

+-----+-----+---------+-----+----+
|   TV|Radio|Newspaper|Sales|cond|
+-----+-----+---------+-----+----+
|230.1| 37.8|     69.2| 22.1|   1|
| 44.5| 39.3|     45.1| 10.4|   3|
| 17.2| 45.9|     69.3|  9.3|   3|
|151.5| 41.3|     58.5| 18.5|   2|
|180.8| 10.8|     58.4| 12.9|   1|
|  8.7| 48.9|       75|  7.2|   3|
| 57.5| 32.8|     23.5| 11.8|   2|
|120.2| 19.6|     11.6| 13.2|   1|
|  8.6|  2.1|        1|  4.8|   3|
|199.8|  2.6|     21.2| 10.6|   1|
| 66.1|  5.8|     24.2|  8.6|   3|
|214.7|   24|        4| 17.4|   1|
| 23.8| 35.1|     65.9|  9.2|   3|
| 97.5|  7.6|      7.2|  9.7|   3|
|204.1| 32.9|       46|   19|   1|
|195.4| 47.7|     52.9| 22.4|   2|
| 67.8| 36.6|      114| 12.5|   2|
|281.4| 39.6|     55.8| 24.4|   1|
| 69.2| 20.5|     18.3| 11.3|   2|
|147.3| 23.9|     19.1| 14.6|   1|
+-----+-----+---------+-----+----+
only showing top 20 rows



In [57]:
ds = ds.withColumn('log_tv',F.log(ds.TV))
ds = ds.withColumn('tv+10', ds.TV+10)
ds.show()

+-----+-----+---------+-----+----+------------------+-----+
|   TV|Radio|Newspaper|Sales|cond|            log_tv|tv+10|
+-----+-----+---------+-----+----+------------------+-----+
|230.1| 37.8|     69.2| 22.1|   1|  5.43851399704132|240.1|
| 44.5| 39.3|     45.1| 10.4|   3|3.7954891891721947| 54.5|
| 17.2| 45.9|     69.3|  9.3|   3|2.8449093838194073| 27.2|
|151.5| 41.3|     58.5| 18.5|   2| 5.020585624949424|161.5|
|180.8| 10.8|     58.4| 12.9|   1|5.1973914479580765|190.8|
|  8.7| 48.9|       75|  7.2|   3| 2.163323025660538| 18.7|
| 57.5| 32.8|     23.5| 11.8|   2| 4.051784947803305| 67.5|
|120.2| 19.6|     11.6| 13.2|   1| 4.789157022101107|130.2|
|  8.6|  2.1|        1|  4.8|   3| 2.151762203259462| 18.6|
|199.8|  2.6|     21.2| 10.6|   1| 5.297316866214453|209.8|
| 66.1|  5.8|     24.2|  8.6|   3| 4.191168746857641| 76.1|
|214.7|   24|        4| 17.4|   1| 5.369241704884735|224.7|
| 23.8| 35.1|     65.9|  9.2|   3| 3.169685580677429| 33.8|
| 97.5|  7.6|      7.2|  9.7|   3| 4.579

### JOIN tromg spark

In [59]:
import pandas as pd
leftp = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                      'B': ['B0', 'B1', 'B2', 'B3'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']},
                     index=[0, 1, 2, 3])

rightp = pd.DataFrame({'A': ['A0', 'A1', 'A6', 'A7'],
                       'F': ['B4', 'B5', 'B6', 'B7'],
                       'G': ['C4', 'C5', 'C6', 'C7'],
                       'H': ['D4', 'D5', 'D6', 'D7']},
                      index=[4, 5, 6, 7])


In [64]:
print(leftp)
print(rightp)



    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1
2  A2  B2  C2  D2
3  A3  B3  C3  D3
    A   F   G   H
4  A0  B4  C4  D4
5  A1  B5  C5  D5
6  A6  B6  C6  D6
7  A7  B7  C7  D7


In [65]:
## Convert to data frame in spark

lefts = spark.createDataFrame(leftp)
rights = spark.createDataFrame(rightp)


In [67]:
lefts.printSchema()

root
 |-- A: string (nullable = true)
 |-- B: string (nullable = true)
 |-- C: string (nullable = true)
 |-- D: string (nullable = true)



In [68]:
lefts.join(rights,on='A',how='left') .orderBy('A',ascending=True).show()

+---+---+---+---+----+----+----+
|  A|  B|  C|  D|   F|   G|   H|
+---+---+---+---+----+----+----+
| A0| B0| C0| D0|  B4|  C4|  D4|
| A1| B1| C1| D1|  B5|  C5|  D5|
| A2| B2| C2| D2|NULL|NULL|NULL|
| A3| B3| C3| D3|NULL|NULL|NULL|
+---+---+---+---+----+----+----+



In [69]:
lefts.join(rights,on='A',how='right') .orderBy('A',ascending=True).show()

+---+----+----+----+---+---+---+
|  A|   B|   C|   D|  F|  G|  H|
+---+----+----+----+---+---+---+
| A0|  B0|  C0|  D0| B4| C4| D4|
| A1|  B1|  C1|  D1| B5| C5| D5|
| A6|NULL|NULL|NULL| B6| C6| D6|
| A7|NULL|NULL|NULL| B7| C7| D7|
+---+----+----+----+---+---+---+



In [70]:
lefts.join(rights,on='A',how='inner') .orderBy('A',ascending=True).show()


+---+---+---+---+---+---+---+
|  A|  B|  C|  D|  F|  G|  H|
+---+---+---+---+---+---+---+
| A0| B0| C0| D0| B4| C4| D4|
| A1| B1| C1| D1| B5| C5| D5|
+---+---+---+---+---+---+---+



In [71]:
lefts.join(rights,on='A',how='full') .orderBy('A',ascending=True).show()


+---+----+----+----+----+----+----+
|  A|   B|   C|   D|   F|   G|   H|
+---+----+----+----+----+----+----+
| A0|  B0|  C0|  D0|  B4|  C4|  D4|
| A1|  B1|  C1|  D1|  B5|  C5|  D5|
| A2|  B2|  C2|  D2|NULL|NULL|NULL|
| A3|  B3|  C3|  D3|NULL|NULL|NULL|
| A6|NULL|NULL|NULL|  B6|  C6|  D6|
| A7|NULL|NULL|NULL|  B7|  C7|  D7|
+---+----+----+----+----+----+----+



In [72]:
my_list = [('a', 2, 3), ('b', 5, 6), ('c', 8, 9), ('a', 2, 3), ('b', 5, 6), ('c', 8, 9)]
col_name = ['col1', 'col2', 'col3']
ds2 = spark.createDataFrame(my_list,schema=col_name)

In [73]:
ds2.show()

+----+----+----+
|col1|col2|col3|
+----+----+----+
|   a|   2|   3|
|   b|   5|   6|
|   c|   8|   9|
|   a|   2|   3|
|   b|   5|   6|
|   c|   8|   9|
+----+----+----+



In [74]:
## Concat
ds2= ds2.withColumn('concat',F.concat('col1','col2'))
ds2.show()

+----+----+----+------+
|col1|col2|col3|concat|
+----+----+----+------+
|   a|   2|   3|    a2|
|   b|   5|   6|    b5|
|   c|   8|   9|    c8|
|   a|   2|   3|    a2|
|   b|   5|   6|    b5|
|   c|   8|   9|    c8|
+----+----+----+------+



In [75]:
## Group by
ds2.groupBy(['col1']).agg({'col2':'min', 'col3':'avg'}).show()

+----+---------+---------+
|col1|min(col2)|avg(col3)|
+----+---------+---------+
|   c|        8|      9.0|
|   b|        5|      6.0|
|   a|        2|      3.0|
+----+---------+---------+



In [77]:
## Order by
ds2.orderBy('col1',ascending = True).show()

+----+----+----+------+
|col1|col2|col3|concat|
+----+----+----+------+
|   a|   2|   3|    a2|
|   a|   2|   3|    a2|
|   b|   5|   6|    b5|
|   b|   5|   6|    b5|
|   c|   8|   9|    c8|
|   c|   8|   9|    c8|
+----+----+----+------+



In [78]:
ds2.groupBy(['col1']).pivot('col2').sum('col3').show()

+----+----+----+----+
|col1|   2|   5|   8|
+----+----+----+----+
|   c|NULL|NULL|  18|
|   b|NULL|  12|NULL|
|   a|   6|NULL|NULL|
+----+----+----+----+

